Gene Read Matching: Parsing SAM/BAM and GTF Files
This repository contains a bioinformatics workflow for parsing SAM/BAM files and comparing them with GTF gene annotation files to determine the number of reads aligned to specific genes. The workflow involves:

Reading and parsing SAM/BAM files (containing aligned sequencing reads).
Comparing the aligned reads to gene annotations (from GTF files), which are sorted by chromosome, exon start, and end positions.
Counting how many reads align to each gene based on the gene's position.
This repository uses example data (not actual experimental data) to demonstrate the workflow, with clear instructions on how to set up the pipeline. The focus is on methods for efficient parsing and comparison between large genomic datasets.

We first install gtfparse which is a Python library that efficiently parses and works with GTF (Gene Transfer Format) files and install pysam which is used to work with SAM (Sequence Alignment/Map), BAM (Binary Alignment/Map).

In [1]:
!pip install git+https://github.com/y9c/gtfparse.git 
!pip install pysam #Installing pysam

  Cloning https://github.com/y9c/gtfparse.git to /tmp/pip-req-build-inl8wwdm
  Running command git clone --filter=blob:none --quiet https://github.com/y9c/gtfparse.git /tmp/pip-req-build-inl8wwdm
  Resolved https://github.com/y9c/gtfparse.git to commit 284a6b04aff70a69c82f939ed6da53322ad3854b
  Preparing metadata (setup.py) ... done
  Created wheel for gtfparse: filename=gtfparse-2.0.2-py3-none-any.whl size=15190 sha256=1f4f337aa9cae53d85a95858b121cce86a320c71e9761c3b5166e1acbf6d68a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-689onrdh/wheels/70/aa/66/3693ca0d87954c0d972a3dc99feff3e14053dd9c97da24976f
Successfully built gtfparse


We import the packages pandas, gtfparse and pysam so these tools can be later used in our analysis.

In [3]:
import pandas as pd #Importing pandas
from gtfparse import *
from pysam import * #Importing everything from pysam

We read the GTF file using the read_gtf command from the gtfparse package. Here, we label our sample data as sample.gtf and assign it to the variable df. Since we only need the exon data from the GTF file, we filter the original dataset and save the filtered data as df_genes.

In [5]:
#Reading the GTF file using the command from the gtfparse package
df = read_gtf("sample.gtf")
#Filtering so we only get the exon data
df_genes = df[df["feature"]=="exon"]
#Looking at the data
df_genes

We then filter the original DataFrame df to keep only exon entries from the first transcript (indicated by .1 in the transcript_id) and check if the filtering reduced the size of the dataset.

In [7]:
#Using regular expression to filter so use only the exons that belong to the first transcript, their ID ends with a .1
exons_data = df[(df["feature"] == "exon") & df["transcript_id"].str.match(r".+\.1$")]

,seqname,source,feature,start,end,score,strand,frame,geneID,gene_id,gene_name,transcript_id
0,Chr1,TAIR10,exon,3631,3913,NaN,+,0,None,AT1G01010,AT1G01010,AT1G01010.1
1,Chr1,TAIR10,exon,3996,4276,NaN,+,0,None,AT1G01010,AT1G01010,AT1G01010.1
2,Chr1,TAIR10,exon,4486,4605,NaN,+,0,None,AT1G01010,AT1G01010,AT1G01010.1
3,Chr1,TAIR10,exon,4706,5095,NaN,+,0,None,AT1G01010,AT1G01010,AT1G01010.1
4,Chr1,TAIR10,exon,5174,5326,NaN,+,0,None,AT1G01010,AT1G01010,AT1G01010.1
...,...,...,...,...,...,...,...,...,...,...,...,...
414335,ChrM,TAIR10,exon,360717,361052,NaN,-,0,None,ATMG01370,ATMG01370,ATMG01370.1
414337,ChrM,TAIR10,exon,361062,361179,NaN,-,0,None,ATMG01380,ATMG01380,ATMG01380.1
414338,ChrM,TAIR10,exon,361350,363284,NaN,-,0,None,ATMG01390,ATMG01390,ATMG01390.1
414339,ChrM,TAIR10,exon,363725,364042,NaN,+,0,None,ATMG01400,ATMG01400,ATMG01400.1


We then open the sample BAM file, iterates over all reads, and extracts specific alignment information (chromosome, alignment start, and BAM file header) for each read, storing these details in reads_data.

In [16]:
#Opening BAM file using instructions from https://pysam.readthedocs.io/en/latest/api.html#sam-bam-cram-files
bam_file = pysam.AlignmentFile("sample.bam", "rb")
#Iterating over the reads to get the chromosome name, and start of alignment and header from the bam file
reads_data = [(read.reference_name, read.reference_start, read.header) for read in bam_file]

The code below goes through each exon in the exons_data DataFrame, counts the number of reads that align to the exon (using the BAM file), and stores the gene names and their corresponding read counts. If a chromosome is not found in the BAM file, the error is handled gracefully, and the script continues processing the next exon.

In [25]:
gene_names = [] #Creating a list of the genes
read_counts = [] #Creating a list of the reads

# Using a for loop to go through iterate through the rows and rename them
for index, row in exons_data.iterrows():
    exon_chrom = row["seqname"]
    exon_start = int(row["start"])
    exon_end = int(row["end"])
    gene_name = row["gene_name"]

    try: #We use try to avoid getting errors when chromosomes would not exist in the BAM file
        # We count to count reads within the exon coordinates
        #Souce: https://pysam.readthedocs.io/en/latest/api.html#pysam.AlignmentFile.count
        count = bam_file.count(exon_chrom, start=exon_start, stop=exon_end)


        # Append gene name and read count to the list
        gene_names.append(gene_name)
        read_counts.append(count)


    except ValueError as e:
        # If we get a Key Error where the chromosome from the gtf does not exist in the BAM file, an error will print
        #and the for loop will continue
        print(f"{exon_chrom} is not found in the BAM file. Skip.")


ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not found in the BAM file. Skip.
ChrC is not foun

The code first creates a DataFrame with gene names and the number of reads. It then groups by gene and sums the read counts for each gene. Finally, it prints the resulting DataFrame, which shows the total number of reads aligned to each gene.

In [26]:
# Creating a DataFrame from the lists using pandas
gene_counts_df = pd.DataFrame({"Gene": gene_names, "Number of Reads": read_counts})

# Group by 'Gene' and sum the 'Number of Reads'
gene_counts_combined = gene_counts_df.groupby('Gene')['Number of Reads'].sum().reset_index()

# Print or use the resulting DataFrame
print(gene_counts_combined)

            Gene  Number of Reads
0      AT1G01010                3
1      AT1G01020                1
2      AT1G01030                0
3      AT1G01040                9
4      AT1G01046                0
...          ...              ...
33263  AT5G67600                0
33264  AT5G67610                0
33265  AT5G67620                0
33266  AT5G67630                2
33267  AT5G67640                1

[33268 rows x 2 columns]
